Will be using this to identify the main electrical faults that have ocurred in the last 25 years or the impact to the structure that has ocurred as a consequence or result

Data from: https://www.ntsb.gov/safety/data/Pages/Data_Stats.aspx

In [1]:
!pip freeze

absl-py==2.1.0
anyio==4.4.0
argon2-cffi==23.1.0
argon2-cffi-bindings==21.2.0
arrow==1.3.0
asttokens==2.4.1
async-lru==2.0.4
attrs==23.2.0
Babel==2.15.0
beautifulsoup4==4.12.3
bleach==6.1.0
certifi==2024.7.4
cffi==1.16.0
charset-normalizer==3.3.2
colorama==0.4.6
comm==0.2.2
contourpy==1.3.0
cycler==0.12.1
debugpy==1.8.2
decorator==5.1.1
defusedxml==0.7.1
docx==0.2.4
executing==2.0.1
fastjsonschema==2.20.0
fonttools==4.53.1
fqdn==1.5.1
h11==0.14.0
h5py==3.11.0
httpcore==1.0.5
httpx==0.27.0
idna==3.7
ipykernel==6.29.5
ipython==8.26.0
ipywidgets==8.1.3
isoduration==20.11.0
jedi==0.19.1
Jinja2==3.1.4
joblib==1.4.2
json5==0.9.25
jsonpointer==3.0.0
jsonschema==4.22.0
jsonschema-specifications==2023.12.1
jupyter==1.0.0
jupyter-console==6.6.3
jupyter-events==0.10.0
jupyter-lsp==2.2.5
jupyter_client==8.6.2
jupyter_core==5.7.2
jupyter_server==2.14.1
jupyter_server_terminals==0.5.3
jupyterlab==4.2.3
jupyterlab_pygments==0.3.0
jupyterlab_server==2.27.2
jupyterlab_widgets==3.0.11
kaleido==0.2.1
kera

In [2]:
import os
import pandas as pd
import numpy as np
import re
import kaleido

In [3]:
if not os.path.exists("images"):
    os.mkdir("images")

In [4]:
print(os.listdir(os.getcwd()))

['.git', '.ipynb_checkpoints', 'aircraft.csv', 'Data Analysis.ipynb', 'Data Preprocessing.ipynb', 'engines.csv', 'findings.csv', 'images', 'README.md', 'seq_events.csv']


In [5]:
def reduce_memory_usage(df, verbose=True):
    numerics = ["int8", "int16", "int32", "int64", "float16", "float32", "float64"]
    start_mem = df.memory_usage().sum() / 1024 ** 2 # in MegaBytes
    for col in df.columns:
        col_type = df[col].dtypes
        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == "int":
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)
            else:
                if (
                    c_min > np.finfo(np.float16).min
                    and c_max < np.finfo(np.float16).max
                ):
                    df[col] = df[col].astype(np.float16)
                elif (
                    c_min > np.finfo(np.float32).min
                    and c_max < np.finfo(np.float32).max
                ):
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)
    end_mem = df.memory_usage().sum() / 1024 ** 2 # in MEgaBytes
    if verbose:
        print(
            f"Mem. usage decreased to {end_mem} Mb",
            f"{100 * (start_mem - end_mem) / start_mem}% reduction)"
        )
    return df

In [6]:
path = r'aircraft.csv'
aircraft_df = pd.read_csv(path, encoding = 'latin-1')
aircraft_df.head(10)

C:\Users\yxb19198\AppData\Local\Temp\ipykernel_34708\2103535336.py:2: DtypeWarning: Columns (34,42,44,49,53) have mixed types. Specify dtype option on import or set low_memory=False.
  aircraft_df = pd.read_csv(path, encoding = 'latin-1')


,ev_id,Aircraft_Key,regis_no,ntsb_no,acft_missing,far_part,flt_plan_filed,flight_plan_activated,damage,acft_fire,...,acft_year,fuel_on_board,commercial_space_flight,unmanned,ifr_equipped_cert,elt_mounted_aircraft,elt_connected_antenna,elt_manufacturer,elt_model,elt_reason_other
0,20080528X00735,1,PT-JGX,NYC08WA175,Y,NUSC,NaN,NaN,NaN,NaN,...,NaN,NaN,False,False,False,False,False,NaN,NaN,NaN
1,20081004X21118,1,LN-SFT,DCA08WA074,N,NaN,NaN,NaN,DEST,NONE,...,NaN,NaN,False,False,False,False,False,NaN,NaN,NaN
2,20081004X11120,1,EI-GBA,DCA08WA086,N,NaN,NaN,NaN,NaN,NONE,...,NaN,NaN,False,False,False,False,False,NaN,NaN,NaN
3,20090210X44213,1,HI-855,CEN09WA160,N,NUSN,NaN,NaN,SUBS,NONE,...,NaN,NaN,False,False,False,False,False,NaN,NaN,NaN
4,20100412X43956,1,NaN,DCA10WA050,N,NUSC,NaN,NaN,DEST,NONE,...,NaN,NaN,False,False,False,False,False,NaN,NaN,NaN
5,20100507X93350,1,G-KPTN,ENG10WA028,N,UNK,NaN,NaN,NaN,NONE,...,NaN,NaN,False,False,False,False,False,NaN,NaN,NaN
6,20100708X84328,1,XB-NVB,CEN10WA374,N,UNK,NaN,NaN,DEST,GRD,...,NaN,NaN,False,False,False,False,False,NaN,NaN,NaN
7,20100901X85159,1,TI-AGR,CEN10RA511,N,UNK,NaN,NaN,SUBS,IFLT,...,NaN,NaN,False,False,False,False,False,NaN,NaN,NaN
8,20100929X01447,1,XB-KYY,CEN10WA574,N,UNK,NaN,NaN,SUBS,NONE,...,NaN,NaN,False,False,False,False,False,NaN,NaN,NaN
9,20101206X53337,1,EC-EJR,CEN11WA096,N,NUSC,NaN,NaN,SUBS,NONE,...,NaN,NaN,False,False,False,False,False,NaN,NaN,NaN


In [7]:
aircraft_df = reduce_memory_usage(aircraft_df, verbose = True)

Mem. usage decreased to 16.835430145263672 Mb 10.578208535833701% reduction)


In [8]:
aircraft_df.dtypes
aircraft_df.size

2589492

Check and remove what columns have no NaNs

Cannot remove all columns that have Nans as they contain the better information
Before altering code - double check if the columns that are left behind have a common root with other csv files

In [9]:
Nan_no_list = list(aircraft_df.isnull().sum())
column_names = list(aircraft_df.columns)
no_col_before = len(column_names)

Columns chosen based on key information - cannot be limited based on number of Nan values as they are not within a specific range of each other or are they?

In [10]:
important_cols = ['ev_id', 'acft_make', 'acft_model', 'acft_series', 'acft_category']
for col in important_cols:
    print(aircraft_df.loc[:, col].isnull().sum())

0
43
54
17342
389


In [11]:
no_col_before

93

In [12]:
for i in range(len(column_names)):
    # if column.isnull() == True then drop the column
    if column_names[i] in important_cols:
        continue
    elif Nan_no_list[i] > 0:
        aircraft_df = aircraft_df.drop(columns = [column_names[i]])
    else:
        continue
no_col_after = len(list(aircraft_df.columns))
print(no_col_after)

18


In [13]:
aircraft_df.columns

Index(['ev_id', 'Aircraft_Key', 'ntsb_no', 'acft_missing', 'acft_make',
       'acft_model', 'acft_series', 'acft_category', 'homebuilt',
       'fixed_retractable', 'oper_individual_name', 'lchg_date',
       'afm_hrs_since', 'commercial_space_flight', 'unmanned',
       'ifr_equipped_cert', 'elt_mounted_aircraft', 'elt_connected_antenna'],
      dtype='object')

lchg_date is last change data to correspond to when the record was last updated - could be a useful measure of how up to date the information is in relation to how documented the accident is from when it occured

afm_hrs_since = hours since the last Airplane Flight Manual (AFM) update or hours since the last recorded maintenance as noted in the AFM

ifr_equipped_cert = boolean if aircraft was certified for Instrument Flight Rules (IFR) operations

ELT_mounted_aircraft = aircraft has Emergency Locator Transmitter (ELT) (usually 406 MHz) which transmit GPS position for search and rescue purposes when activated in cockpit

elt_conected antenna must therefore aline unless the error occurs that it was not activated due to immediate catastrophic mode

Conjoin different excel data files to each other based on same ev_ids

Based on an inner join there may be some data excluded if the same accident isn't included in the other excel sheet. Joining two dfs is done using merge in pandas.

In [14]:
path = r'engines.csv'
engines_df = pd.read_csv(path, encoding = 'latin-1')
engines_df.head(10)

,ev_id,Aircraft_Key,eng_no,eng_type,eng_mfgr,eng_model,power_units,hp_or_lbs,lchg_userid,lchg_date,carb_fuel_injection,propeller_type,propeller_make,propeller_model,eng_time_total,eng_time_last_insp,eng_time_overhaul
0,20080630X00944,1,1,REC,Lycoming,O-320 SERIES,180.0,HP,NaN,25/09/2020 18:06,CARB,NaN,NaN,NaN,0,0,0
1,20080810X01199,1,1,REC,Lycoming,O-320 SERIES,180.0,HP,NaN,25/09/2020 18:06,CARB,NaN,NaN,NaN,0,0,0
2,20080917X01489,1,1,REC,Lycoming,O-320 SERIES,180.0,HP,NaN,25/09/2020 18:04,CARB,NaN,NaN,NaN,0,0,0
3,20081004X12035,1,1,REC,Lycoming,O-320 SERIES,180.0,HP,NaN,03/11/2020 20:28,CARB,NaN,NaN,NaN,0,0,0
4,20081003X62315,1,1,REC,Lycoming,O-320 SERIES,180.0,HP,NaN,25/09/2020 18:04,CARB,NaN,NaN,NaN,0,0,0
5,20081003X62316,1,1,REC,Lycoming,O-320 SERIES,180.0,HP,NaN,25/09/2020 18:04,CARB,NaN,NaN,NaN,0,0,0
6,20081021X92821,1,1,REC,Lycoming,O-320 SERIES,180.0,HP,NaN,25/09/2020 18:04,CARB,NaN,NaN,NaN,0,0,0
7,20081027X12391,1,1,REC,Lycoming,O-320 SERIES,180.0,HP,NaN,03/11/2020 20:28,CARB,NaN,NaN,NaN,0,0,0
8,20081110X43253,1,1,REC,Lycoming,O-320 SERIES,180.0,HP,NaN,25/09/2020 18:04,CARB,NaN,NaN,NaN,0,0,0
9,20081117X62306,1,1,REC,Lycoming,O-320 SERIES,180.0,HP,NaN,25/09/2020 18:04,CARB,NaN,NaN,NaN,0,0,0


In [15]:
set(engines_df.loc[:, 'eng_type'])

{'ELEC', 'GTFN', 'HR', 'LR', 'NONE', 'REC', 'TF', 'TJ', 'TP', 'TS', 'UNK', nan}

### Engine types explained: 
GTFN: Might be referring to GTF which is a Geared Turbofan Engine <br>
HR (Hysteresis Motor): A type of synchronous motor that uses the hysteresis effect in a magnetic material to produce torque2. <br>
LR (Linear Motor): Produces linear motion instead of rotational motion. Often used in applications like maglev trains and conveyor systems2. <br>
NONE: Indicates no motor is present or required in the context. <br>
REC: Reciprocating engine typically used by small propeller-driven planes <br>
TF (Torque Motor): Designed to provide high torque at low speeds, often used in applications requiring precise control2. <br>
TJ: Turbojet engine <br>
TP (Two-Phase Motor): A type of AC motor that operates on two-phase power, often used in older systems2. <br>
TS (Three-Phase Motor): A type of AC motor that operates on three-phase power, commonly used in industrial applications for its efficiency and power2. <br>
UNK (Unknown): Indicates the motor type is unknown or not specified. 

In [16]:
column_names = list(engines_df.columns)
no_col_before = len(engines_df.columns)
important_cols = ['ev_id', 'eng_type', 'eng_mfgr', 'eng_model', 'power_units']
for i in range(len(column_names)):
    # if column.isnull() == True then drop the column
    if column_names[i] in important_cols:
        continue
    elif Nan_no_list[i] > 0:
        engines_df = engines_df.drop(columns = [column_names[i]])
    else:
        continue
no_col_after = len(list(engines_df.columns))
print(no_col_after)

6


In [17]:
engines_df.head(5)

,ev_id,Aircraft_Key,eng_type,eng_mfgr,eng_model,power_units
0,20080630X00944,1,REC,Lycoming,O-320 SERIES,180.0
1,20080810X01199,1,REC,Lycoming,O-320 SERIES,180.0
2,20080917X01489,1,REC,Lycoming,O-320 SERIES,180.0
3,20081004X12035,1,REC,Lycoming,O-320 SERIES,180.0
4,20081003X62315,1,REC,Lycoming,O-320 SERIES,180.0


## Findings

In [18]:
path = r'findings.csv'
findings_df = pd.read_csv(path, encoding = 'latin-1')
findings_df.head(10)

,ev_id,Aircraft_Key,finding_no,finding_code,finding_description,category_no,subcategory_no,section_no,subsection_no,modifier_no,Cause_Factor,lchg_date,lchg_userid,cm_inPc
0,20160118X53003,1,2,206304044,Personnel issues-Task performance-Use of equip...,2,6,30,40,44,C,25/09/2020 14:28,NaN,T
1,20160216X81446,1,2,206304044,Personnel issues-Task performance-Use of equip...,2,6,30,40,44,C,25/09/2020 14:28,NaN,T
2,20160302X11902,1,2,206304044,Personnel issues-Task performance-Use of equip...,2,6,30,40,44,C,25/09/2020 14:28,NaN,T
3,20160307X85243,1,2,206304044,Personnel issues-Task performance-Use of equip...,2,6,30,40,44,C,25/09/2020 14:28,NaN,T
4,20160312X04625,1,2,206304044,Personnel issues-Task performance-Use of equip...,2,6,30,40,44,C,25/09/2020 14:28,NaN,T
5,20160312X21414,1,2,206304044,Personnel issues-Task performance-Use of equip...,2,6,30,40,44,C,25/09/2020 14:28,NaN,T
6,20160323X12813,1,2,206304044,Personnel issues-Task performance-Use of equip...,2,6,30,40,44,C,25/09/2020 14:28,NaN,T
7,20160325X80001,1,2,206304044,Personnel issues-Task performance-Use of equip...,2,6,30,40,44,C,25/09/2020 14:28,NaN,T
8,20160329X75123,1,2,206304044,Personnel issues-Task performance-Use of equip...,2,6,30,40,44,C,25/09/2020 14:28,NaN,T
9,20160416X61819,1,2,206304044,Personnel issues-Task performance-Use of equip...,2,6,30,40,44,C,25/09/2020 14:28,NaN,T


In [19]:
# What is the difference bwteen cause factors?
set(findings_df.loc[:, 'Cause_Factor'])

{'C', 'F', nan}

In [20]:
# To start only the finding_description will be used
findings_df = findings_df[['ev_id', 'finding_code', 'finding_description']]
print('The number of different fault scenarios is', len(set(findings_df.loc[:, 'finding_code'])))

The number of different fault scenarios is 3989


## Seq_Events
Gives an indication of how the aircraft landed.

In [21]:
path = r'seq_events.csv'
seq_events_df = pd.read_csv(path, encoding = 'latin-1')
seq_events_df.head(10)

,ev_id,Aircraft_Key,Occurrence_No,Occurrence_Code,Occurrence_Description,phase_no,eventsoe_no,Defining_ev,lchg_date,lchg_userid
0,20080220X00212,1,1,552230,Landing-landing roll Loss of control on ground,552,230,True,25/09/2020 18:06,NaN
1,20080115X00052,1,1,552230,Landing-landing roll Loss of control on ground,552,230,True,25/09/2020 18:05,NaN
2,20080214X00188,1,1,552230,Landing-landing roll Loss of control on ground,552,230,True,25/09/2020 18:06,NaN
3,20080321X00361,1,1,552230,Landing-landing roll Loss of control on ground,552,230,True,25/09/2020 18:04,NaN
4,20080222X00230,1,1,552230,Landing-landing roll Loss of control on ground,552,230,True,25/09/2020 18:04,NaN
5,20080306X00278,1,1,552230,Landing-landing roll Loss of control on ground,552,230,True,25/09/2020 18:05,NaN
6,20080306X00279,1,1,552230,Landing-landing roll Loss of control on ground,552,230,True,25/09/2020 18:07,NaN
7,20080326X00388,1,1,552230,Landing-landing roll Loss of control on ground,552,230,True,25/09/2020 18:05,NaN
8,20080318X00329,1,1,552230,Landing-landing roll Loss of control on ground,552,230,True,25/09/2020 18:06,NaN
9,20080326X00387,1,1,552230,Landing-landing roll Loss of control on ground,552,230,True,25/09/2020 18:05,NaN


In [22]:
seq_events_df = seq_events_df[['ev_id', 'Occurrence_Code', 'Occurrence_Description']]
print('The number of different ocurrences is', len(set(seq_events_df.loc[:, 'Occurrence_Code'])))

The number of different ocurrences is 1550


## Compile all

In [23]:
merged_df = pd.merge(left=aircraft_df, right=engines_df, left_on='ev_id', right_on='ev_id')
merged_df = pd.merge(left=merged_df, right=findings_df, left_on='ev_id', right_on='ev_id')
merged_df = pd.merge(left=merged_df, right=seq_events_df, left_on='ev_id', right_on='ev_id')
merged_df.head(10)

,ev_id,Aircraft_Key_x,ntsb_no,acft_missing,acft_make,acft_model,acft_series,acft_category,homebuilt,fixed_retractable,...,elt_connected_antenna,Aircraft_Key_y,eng_type,eng_mfgr,eng_model,power_units,finding_code,finding_description,Occurrence_Code,Occurrence_Description
0,20120424X44350,1,CEN11WA690,N,PIPER,PA 28,NaN,NaN,N,FIXD,...,False,1,REC,Lycoming,IO-360,NaN,500000098,Not determined-Not determined-(general)-(gener...,450341,Maneuvering Loss of engine power (total)
1,20120815X72239,1,ANC12WA087,N,PIPER,PA-30,NaN,AIR,N,RETR,...,False,1,REC,Lycoming,IO-320,160.0,500000098,Not determined-Not determined-(general)-(gener...,400000,Enroute Unknown or undetermined
2,20120815X72239,1,ANC12WA087,N,PIPER,PA-30,NaN,AIR,N,RETR,...,False,1,REC,NaN,IO-320,160.0,500000098,Not determined-Not determined-(general)-(gener...,400000,Enroute Unknown or undetermined
3,20081003X17550,1,LAX08WA133,N,BELL,206L1,NaN,HELI,N,FIXD,...,False,1,TS,Rolls-Royce,250-C30P,250.0,500000098,Not determined-Not determined-(general)-(gener...,500440,Approach Off-field or emergency landing
4,20100216X00010,1,CEN10WA121,N,Cessna,550B,CITATION I,AIR,N,FIXD,...,False,1,NaN,Pratt and Whitney,PW530A,NaN,500000098,Not determined-Not determined-(general)-(gener...,401240,Enroute-climb to cruise Loss of control in flight
5,20100216X00010,1,CEN10WA121,N,Cessna,550B,CITATION I,AIR,N,FIXD,...,False,1,NaN,Pratt and Whitney,PW530A,NaN,500000098,Not determined-Not determined-(general)-(gener...,451240,Maneuvering-aerobatics Loss of control in flight
6,20100216X00010,1,CEN10WA121,N,Cessna,550B,CITATION I,AIR,N,FIXD,...,False,1,NaN,Pratt and Whitney,PW530A,NaN,500000098,Not determined-Not determined-(general)-(gener...,401240,Enroute-climb to cruise Loss of control in flight
7,20100216X00010,1,CEN10WA121,N,Cessna,550B,CITATION I,AIR,N,FIXD,...,False,1,NaN,Pratt and Whitney,PW530A,NaN,500000098,Not determined-Not determined-(general)-(gener...,451240,Maneuvering-aerobatics Loss of control in flight
8,20090121X45440,1,CEN09WA138,N,PIPER,PA-42-720,NaN,AIR,N,RETR,...,False,1,TP,Pratt and Whitney Canada,NaN,NaN,500000098,Not determined-Not determined-(general)-(gener...,402120,Enroute-cruise Controlled flight into terr/obj...
9,20090121X45440,1,CEN09WA138,N,PIPER,PA-42-720,NaN,AIR,N,RETR,...,False,1,TP,Pratt and Whitney Canada,NaN,NaN,500000098,Not determined-Not determined-(general)-(gener...,402120,Enroute-cruise Controlled flight into terr/obj...


In [24]:
# Save the merged df to then reaccess
merged_df.to_pickle("US_Accidents_Processed_df.pkl")